In [1]:
import scipy.optimize as opt
import numpy as np
import pandas as pd
import scipy.stats as stats

In [2]:
df_likelihood = pd.read_excel(
    "data/Data_2024.xlsx", sheet_name="Data_věrohodnost")

df_likelihood = df_likelihood.rename(
    columns={"doba práce v oboru [roky]": "doba"})

data_censored = df_likelihood.loc[df_likelihood["censored"] == 1, "doba"]
data_uncensored = df_likelihood.loc[df_likelihood["censored"] == 0, "doba"]

In [3]:
def neg_log_likelihood(params, data_unc, data_c):
    k, lambd = params
    if k <= 0 or lambd <= 0:
        return np.inf

    # cenzorovana data
    term1 = np.sum(np.power(data_c / lambd, k))

    # necenzorovana data
    n = data_unc.size
    term2 = n * np.log(k) - n * k * np.log(lambd) + (k - 1) * \
        np.sum(np.log(data_unc)) - np.sum(np.power(data_unc / lambd, k))

    llh = -term1 + term2
    return -llh


result = opt.minimize(neg_log_likelihood, x0=[1.0, 1.0], args=(
    data_uncensored, data_censored), bounds=[(0, None), (0, None)])

k_mle = result.x[0]
lambda_mle = result.x[1]

In [ ]:
# l0(1, lambda_mle)

n = data_uncensored.size
l0 = -np.sum(data_censored / lambda_mle) - n * \
    np.log(lambda_mle) - np.sum(data_uncensored / lambda_mle)

print(f"l0 = {l0}")

# l0(k_mle, lambda_mle)

l1 = -np.sum(np.power(data_censored / lambda_mle, k_mle)) + n * np.log(k_mle) - n * k_mle * np.log(lambda_mle) + \
    (k_mle - 1) * np.sum(np.log(data_uncensored)) - \
    np.sum(np.power(data_uncensored / lambda_mle, k_mle))

print(f"l1 = {l1}")

lr = -2 * (l0 - l1)

critical_value = stats.chi2.ppf(0.95, df=1)
print(f"lr = {lr}, kriticky obor = (0, {critical_value})")

if lr > critical_value:
    print("H0 zamítnuta")
else:
    print("H0 nezamítnuta")

l0 = -751.1964661007985
l1 = -450.1339063894518
lr = 602.1251194226934, kriticky obor = (0, 3.841458820694124)
H0 zamítnuta


In [ ]:
mean = stats.weibull_min.mean(c=k_mle, scale=lambda_mle)
q_0_1 = stats.weibull_min.ppf(0.1, c=k_mle, scale=lambda_mle)

print(f"Střední hodnota: {mean}")
print(f"0.1 kvantil: {q_0_1}")

Střední hodnota: 6.903243444764023
0.1 kvantil: 5.159770582413018
